In [1]:
%load_ext autotime
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/magellan/')

This quickstart guide explains how to match two tables using Magellan. Our goal is to match DBLP and ACM datasets [add reference] containing information about the conference papers published in top databse conferences. Specifically, we want to achieve precision greater than 95% and get recall as high as possible.

First, we need to import the Magellan package as follows:

In [2]:
# Import libraries
import magellan as mg
import pandas as pd
import os, sys

time: 1.65 s


In [4]:
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + mg.__version__ )

python version: 2.7.12 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:43:17) 
[GCC 4.2.1 (Based on Apple Inc. build 5658) (LLVM build 2336.11.00)]
pandas version: 0.18.1
magellan version: 0.1.0
time: 2.04 ms


Matching two tables typically consists of the following three steps:
1. Loading the input tables
2. Blocking the input tables to get a candidate set
3. Matching the tuple pairs in the candidate set

# 1. Loading the input tables

We begin by loading the input tables. For the purpose of this guide, we use the dataset that is included with the package.

In [5]:
dblp_dataset_path = os.sep.join(['DBLP_ACM', 'DBLP_cleaned.csv'])
acm_dataset_path = os.sep.join(['DBLP_ACM', 'ACM_cleaned.csv'])

time: 1.56 ms


In [6]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = mg.read_csv_metadata(dblp_dataset_path, key='id')
B = mg.read_csv_metadata(acm_dataset_path, key='id')

time: 29 ms


In [7]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2616
Number of tuples in B: 2294
Number of tuples in A X B (i.e the cartesian product): 6001104
time: 2 ms


In [8]:
A.head(2)

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996


time: 25.7 ms


In [9]:
B.head(2)

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999


time: 14.4 ms


In [10]:
# Display the key attributes of table A and B.
mg.get_key(A), mg.get_key(B)

('id', 'id')

time: 3.49 ms


# 2. Blocking to create candidate tuple pairs

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching. 

Magellan provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. Refer to [api reference] for more details. The user can mix an match these blockers to form a blocking sequence 
applied to input tables.

For the matching problem at hand, we know that two conference papers published in different years cannot match. So, we decide to apply an attribute equivelance blocker on the 'year' attribute for this purpose. 

In [11]:
# Plan
# A, B ------ attribute equivalence [year] -----> C1

time: 825 µs


In [12]:
# Create attribute equivalence blocker
ab = mg.AttrEquivalenceBlocker()
# Block tables using 'year' attribute: same year then include in the canidate set
C1 = ab.block_tables(A, B, 'year', 'year', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

time: 265 ms


In [13]:
# Check the number of rows in C1
len(C1)

601284

time: 2.85 ms


In [14]:
# Display first two rows from C1
C1.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/sigmod/Mackay99,304586,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",1999
1,1,journals/sigmod/Mackay99,304587,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",1999


time: 27.8 ms


The number of tuple pairs considered for matching is reduced to 601284 (from 6001104), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in Magellan as follows:

In [15]:
# Debug blocker output
dbg = mg.debug_blocker(A, B, C1, output_size=200)

time: 3.37 s


In [16]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head()

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.875,journals/sigmod/Aberer02,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
1,1,0.875,journals/sigmod/Aberer02a,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
2,2,0.875,journals/sigmod/Aberer02b,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
3,3,0.875,journals/sigmod/Aberer03b,601865,Book review column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
4,4,0.875,journals/sigmod/Aberer02,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record


time: 25.8 ms


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the conference papers to match the author names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [17]:
# Updated blocking sequence
# A, B ------ attribute equivalence [year] -----> C1--
#                                                     |----> C
# A, B ------ overlap blocker [authors] --------> C2--

time: 986 µs


In [18]:
# Create an overlap blocker
ob = mg.OverlapBlocker()
# Apply overlap blocker on 'authors' attribute
C2 = ob.block_tables(A, B, 'authors', 'authors', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:02


time: 2.74 s


In [19]:
# Check the number of rows in C2
len(C2)

307217

time: 3.21 ms


In [20]:
# Display first two rows from C2
C2.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/tods/LechtenborgerV03,304586,On the computation of relational view complements,"Vossen, Jens Gottfried Lechtenbrger",2003,The WASA2 object-oriented workflow management system,"Mathias Vossen, Gottfried Weske",1999
1,1,journals/sigmod/CherniakV03,304586,Reminiscences on Influential Papers,"Mitch Kenneth Cherniack, Gottfried A. Vossen Ross,",2003,The WASA2 object-oriented workflow management system,"Mathias Vossen, Gottfried Weske",1999


time: 18.1 ms


In [21]:
# Combine blocker outputs
C = mg.combine_blocker_outputs_via_union([C1, C2])

time: 1.26 s


In [22]:
# Check the number of rows in the consolidated candidate set.
len(C)

875758

time: 3 ms


We observe the number of tuple pairs considered for matching is increased to 875758 (from 601284). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [23]:
# Debug again
dbg = mg.debug_blocker(A, B, C)

time: 5.94 s


In [24]:
# Display first few rows from the debugger output
dbg.head(3)

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.500000,journals/sigmod/Dogac02,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,ACM SIGMOD Record
1,1,0.500000,journals/sigmod/Dogac98,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,ACM SIGMOD Record
2,2,0.461538,journals/sigmod/Snodgrass98a,641001,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,ACM SIGMOD Record


time: 24.6 ms


We observe that the current blocker sequence does not drop potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.

# 3. Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set, obtained in the previous step. Specifically, we use learning-based method for matching purposes.

This typically involoves the following five steps:

1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set
5. Applying the selected matcher to the candidate set

## 3.1 Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [25]:
# Sample candidate set
S = mg.sample_table(C, 450)

time: 1.44 s


Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [26]:
# Label S and specify the attribute name for the label column
L = mg.label_table(S, 'gold')

time: 11.2 s


For the purposes of this guide, we will load in a pre-labeled dataset included in this package.

In [27]:
# Load the pre-labeled data
L = mg.read_csv_metadata('DBLP_ACM/dblp_acm_demo_labels.csv', ltable=A, rtable=B)
# Display the number of rows in the labaled data set
len(L)

415

time: 10.5 ms


## 3.2 Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development and evaluation. Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data (i.e the evaluation set).

In [28]:
# Split the labeled data into development and evaluation set
development_evaluation = mg.split_train_test(L, train_proportion=0.7)
development =  development_evaluation['train']
evaluation = development_evaluation['test']

time: 11.6 ms


## 3.3 Select the best learning-based matcher

Selecting the best learning-based matcher typically involves the following steps:
1. Creating a set of learning-based matchers
2. Creating features
3. Extracting feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation
5. Debugging the matcher (and possibly repeat the above steps)

### 3.3.1 Creating a set of learning-based matchers

First, we need to create a set of learning-based matchers. The following matchers are supported in Magellan: (1) decision tree, (2) random forest, (3) naive bayes, (4) svm, (5) logistic regression, and (6) linear regression. Refer [api ref] for more details. 

In [29]:
# Create a set of ML-matchers
dt = mg.DTMatcher(name='DecisionTree')
svm = mg.SVMMatcher(name='SVM')
rf = mg.RFMatcher(name='RF')
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

time: 3.66 ms


### 3.3.2 Creating features

Next, we need to create a set of features for the development set. Magellan provides a way to automatically generate features based on the statistical characteristics of the input tables. For the purpose of this guide, we use the automatically generated features.

In [30]:
# Generate features
feature_table = mg.get_features_for_matching(A, B)

time: 67.6 ms


In [31]:
# List the names of the features generated
feature_table['feature_name']

0             title_title_jac_qgm_3_qgm_3
1         title_title_cos_dlm_dc0_dlm_dc0
2                         title_title_mel
3                    title_title_lev_dist
4                     title_title_lev_sim
5         authors_authors_jac_qgm_3_qgm_3
6     authors_authors_cos_dlm_dc0_dlm_dc0
7                     authors_authors_mel
8                authors_authors_lev_dist
9                 authors_authors_lev_sim
10                          year_year_exm
11                          year_year_anm
12                     year_year_lev_dist
13                      year_year_lev_sim
Name: feature_name, dtype: object

time: 5.28 ms


We observe that there were 14 features generated. As a first step, lets say that we decide to use only year related features.

In [32]:
# Select the author related features
feature_subset_iter1 = feature_table[10:14]

time: 1.6 ms


In [33]:
# List the names of the features selected
feature_subset_iter1['feature_name']

10         year_year_exm
11         year_year_anm
12    year_year_lev_dist
13     year_year_lev_sim
Name: feature_name, dtype: object

time: 4.83 ms


### 3.3.3 Extracting feature vectors

In this step, we extract feature vectors using the development set and the created features.

In [37]:
# Extract feature vectors
feature_vectors_dev = mg.extract_feature_vecs(development, 
                            feature_table=feature_subset_iter1, 
                            attrs_after='gold')                                         

0%                          100%
[##############################] | ETA: 00:00:00

time: 323 ms



Total time elapsed: 00:00:00


In [38]:
# Display first few rows
feature_vectors_dev.head(3)

,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold
16,16,conf/sigmod/BonnetBCCCHKMMOSS99,765531,0,0.999500,1.0,0.75,0
78,78,conf/sigmod/LeeWMS01,375709,1,1.000000,0.0,1.00,1
212,212,conf/vldb/ManegoldBK02,765565,0,0.997502,4.0,0.00,0


time: 18.5 ms


Next, we may have to impute the feature vectors as it may contain missing values. First, let us check if there are any missing values in the extracted feature vectors.

In [39]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_dev))

True

time: 4.29 ms


We observe that the extracted feature vectors contain missing values. We have to impute the missing values for the learning-based matchers to fit the model correctly. For the purposes of this guide, we impute the missing value in a column with the mean of the values in that column. 

In [40]:
# Impute feature vectors with the mean of the column values.
feature_vectors_dev = mg.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

time: 13.6 ms


### 3.3.4 Selecting the best matcher using cross-validation

Now, we select the best matcher using cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' metric to select the best matcher.

In [41]:
# Select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='precision') 

time: 276 ms


In [42]:
# Check the cross validation statistics
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object at 0x111296650>,5,0.704545,0.688889,0.833333,0.780488,0.809524,0.763356
1,RF,<magellan.matcher.rfmatcher.RFMatcher object at 0x111296750>,5,0.725000,0.847826,0.785714,0.682927,0.755556,0.759405
2,SVM,<magellan.matcher.svmmatcher.SVMMatcher object at 0x111296690>,5,0.765957,0.769231,0.674419,0.825000,0.777778,0.762477
3,NB,<magellan.matcher.nbmatcher.NBMatcher object at 0x1112967d0>,5,0.727273,0.720930,0.731707,0.785714,0.840909,0.761307
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x111296890>,5,0.769231,0.804878,0.717391,0.818182,0.704545,0.762845
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x111296910>,5,0.790698,0.795455,0.763158,0.690476,0.765957,0.761149


time: 25.5 ms


### 3.3.5 Debugging matcher

We observe that the best matcher is not getting us to the precision that we expect (i.e > 95%). We debug the matcher to see what might be wrong.

To do this, first we split the feature vectors into train and test.

In [43]:
# # Split feature vectors into train and test
train_test = mg.split_train_test(feature_vectors_dev, train_proportion=0.5)
train = train_test['train']
test = train_test['test']

time: 10.6 ms


Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher debugging purposes.

In [44]:
# Debug decision tree using GUI
mg.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold')

time: 6.85 s


From the GUI, we observe that using only year related features result in a lot of false positives. So we decide to all features in the feature table, which had author, title and year related features.

In [45]:
# Select all features from the feature table
feature_subset_iter2 = feature_table

time: 1.1 ms


Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [65]:
# Get new set of features
feature_vectors_dev = mg.extract_feature_vecs(development, feature_table=feature_subset_iter2, attrs_after='gold')

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


time: 1.05 s


In [67]:
# Check if imputation is required
any(pd.isnull(feature_vectors_dev))

True

time: 3.78 ms


In [68]:
# Impute feature vectors
feature_vectors_dev = mg.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

time: 15.2 ms


In [69]:
# Apply cross validation to find if there is a better matcher
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold', metric='f1') 

time: 239 ms


In [70]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object at 0x111296650>,5,0.982456,0.985915,0.961039,0.933333,1.000000,0.972549
1,RF,<magellan.matcher.rfmatcher.RFMatcher object at 0x111296750>,5,0.967742,1.000000,1.000000,0.985915,1.000000,0.990731
2,SVM,<magellan.matcher.svmmatcher.SVMMatcher object at 0x111296690>,5,0.906250,0.949153,0.950820,0.965517,0.957746,0.945897
3,NB,<magellan.matcher.nbmatcher.NBMatcher object at 0x1112967d0>,5,0.985075,0.984127,0.983607,1.000000,0.985075,0.987577
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x111296890>,5,1.000000,0.968750,0.947368,0.987013,0.985507,0.977728
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x111296910>,5,0.944444,0.983051,1.000000,1.000000,0.985507,0.982601


time: 23.5 ms


Now, observe the best matcher is achieving the expected precision and we can proceed on to evaluating the best matcher on the unseen data (the evaluation set).

## 3.4 Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Extracting the feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### 3.4.1 Extracting the feature vectors

As before, we extract the feature vectors (using the updated feature table and the evaluation set) and impute it (if necessary).

In [71]:
# Get new set of features
feature_vectors_eval = mg.extract_feature_vecs(evaluation, 
                                               feature_table=feature_subset_iter2, 
                                               attrs_after='gold')


0%                          100%
[##############################] | ETA: 00:00:00

time: 548 ms



Total time elapsed: 00:00:00


In [72]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_eval))

True

time: 4.4 ms


In [73]:
# Impute feature vectors
feature_vectors_eval = mg.impute_table(feature_vectors_eval, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

time: 14.7 ms


### 3.4.2 Training the matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [74]:
# Train using feature vectors from the development set
rf.fit(table=feature_vectors_dev, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')


time: 46.6 ms


### 3.4.3 Predicting the matches
Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [75]:
# Predict M 
predictions = rf.predict(table=feature_vectors_eval, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

time: 5.93 ms


### 3.4.4 Evaluating the matching output

Finally, we evaluate the predicted outputs

In [76]:
# Evaluate the result
eval_result = mg.eval_matches(predictions, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 98.55% (68/69)
Recall : 100.0% (68/68)
F1 : 99.27%
False positives : 1 (out of 69 positive predictions)
False negatives : 0 (out of 56 negative predictions)
time: 14.9 ms


## 3.5 Applying the matcher to candidate set

This typically involves the following steps:
1. Extracting feature vectors for the candidate set
2. Training the matcher using all of labeled data
3. Predicting the matches for the canidate set

The above three steps are similar to 'evaluating the selected matcher', except we dont have an evaluation step.

### 3.5.1 Extracting feature vectors for the candidate set

First, we need to extract feature vectors using the updated feature table. For exposition, we take a sample of 1000 tuples of candidate set and extract feature vectors from it.

In [89]:
C.columns

Index([u'_id', u'ltable_id', u'rtable_id', u'ltable_title', u'ltable_authors',
       u'ltable_year', u'rtable_title', u'rtable_authors', u'rtable_year'],
      dtype='object')

time: 3.76 ms


In [90]:
# Sample 1000 tuples from the candidate set. This is done just for exposition.
M = mg.sample_table(C, 1000)

# Extract feature vectors
feature_vectors_candset = mg.extract_feature_vecs(M, 
                                                  attrs_before=[u'ltable_title', u'ltable_authors',
                                                   u'ltable_year', u'rtable_title', u'rtable_authors', u'rtable_year'],
                                                  feature_table=feature_subset_iter2)


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:02


time: 4.23 s


In [91]:
any(pd.isnull(feature_vectors_candset))

True

time: 5.18 ms


In [92]:
# Impute feature vectors
feature_vectors_candset = mg.impute_table(feature_vectors_candset, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 
                              u'ltable_title', u'ltable_authors',
                              u'ltable_year', u'rtable_title', u'rtable_authors', u'rtable_year'],
                strategy='mean')

ValueError: could not convert string to float: Georges Gardarin, Fernando Machuca, Philippe Pucheral

time: 48.8 ms


### 3.5.2 Training the matcher using all of labeled data

Next, we train the matcher using all of labeled data

In [86]:
feature_vectors_lbl = mg.extract_feature_vecs(L, feature_table=feature_table, 
                           attrs_after='gold')
# Impute feature vectors
feature_vectors_lbl = mg.impute_table(feature_vectors_lbl, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')
rf.fit(table=feature_vectors_lbl,
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


time: 1.42 s


### 3.5.3 Predicting the matches for the candidate set

Finally, we predict the matches for the (sampled) candidate set.

In [87]:
# Predict (sampled) cand.set
predictions = rf.predict(table=feature_vectors_candset, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

time: 6.8 ms


In [88]:
predictions.head()

,_id,ltable_id,rtable_id,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,authors_authors_jac_qgm_3_qgm_3,authors_authors_cos_dlm_dc0_dlm_dc0,authors_authors_mel,authors_authors_lev_dist,authors_authors_lev_sim,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,predicted
1108,1108,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,190642,0.033708,0.000000,0.608364,35.0,0.285714,0.100877,0.075593,0.566446,199.0,0.145923,0.0,0.995507,4.0,0.00,0
1139,1139,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,195666,0.035294,0.000000,0.573486,32.0,0.304348,0.020833,0.075593,0.537224,211.0,0.094421,0.0,0.995507,4.0,0.00,0
2156,2156,conf/sigmod/AbiteboulBCMM03,945729,0.009709,0.000000,0.552657,45.0,0.181818,0.037234,0.000000,0.617371,97.0,0.211382,1.0,1.000000,0.0,1.00,0
3254,3254,conf/sigmod/AcharyaFZ97,187449,0.000000,0.000000,0.444114,36.0,0.142857,0.044728,0.053300,0.538282,258.0,0.128378,0.0,0.998498,1.0,0.75,0
3412,3412,conf/sigmod/AcharyaFZ97,262771,0.027778,0.104828,0.553824,87.0,0.209091,0.010417,0.000000,0.522048,45.0,0.166667,1.0,1.000000,0.0,1.00,0


time: 40.9 ms
